##  Optimze and a Dense Neural Network for gap filling and feature identification

** With a few tweaks to RepRunner, an LSTM can be run instead

In [1]:
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
# from functools import partial
from multiprocessing import Pool
from sklearn import metrics
# from sklearn.model_selection import train_test_split

## Personal Modules
import ReadStandardTimeFill as RSTF
import importlib
import DenseNet as Dense
import MiscFuncs as MF
importlib.reload(Dense)
importlib.reload(RSTF)
importlib.reload(MF)

# %matplotlib inline

%matplotlib notebook
%config IPCompleter.greedy=True

from scipy.optimize import minimize, curve_fit
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.externals import joblib
from matplotlib import cm

from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

from ipywidgets import FloatProgress
from IPython.display import display
import os  
from functools import partial
import shutil
from keras import backend as K
try:pool.close()
except:pass


def Test(params,X,y,YScaled,XScaled,pool):
    return(np.random.rand(params['K']))


def ModSelect(Scope,Site):
    if Site == 'Illisarvik':
        if Scope == 'Full':
            Model = ['H','wind_speed','air_pressure','PPFD_Avg','AirTC_Avg','VPD',
                    'Temp','VWC','Sedge','Shrub','Grass','Sparse','Out_of_Basin']
        if Scope == 'Test':
            Model = ['PPFD_Avg','wind_speed']#,'Temp','VWC','Sedge']
    if Site == 'FishIsland':
        BaseFactors = []
        if Scope == 'Full':
            Model = ['H','Wind Spd','air pressure','Ta','Rn','PPFD','Rain','Water Table',
            'Ts 2.5 cm','Ts 15 cm','VWC','Active Layer','24H Rain','Wtr Tbl Trnd']
        if Scope == 'Test':
            Model = ['H','Water Table','Wind Spd','Active Layer']
    return(Model)

def Combos(Model,L,factor=None):
    Models=[]
    for c in combinations(Model,L):
        c = list(c)
        if factor is None:
            Models.append(c)
        else:
            for f in factor:
                f = f.split('+')
                if set(f).issubset(set(c)) and c not in Models:
                    Models.append(c)
                    
    print('Models: ',Models)
    return(Models)

def Stats(mse,j,i,params):
    df = pd.DataFrame(index = [str(j)+'_'+str(i)],
                      data={'Model':[params['Model']],
                            'MSE':[mse.mean()],
                            'SE':[mse.std()/params['K']**.5],
                            'Performance':0})
    return(df)

def t(p,n):
    alpha = 1-p
    df = n-1
    return(stats.t.ppf(alpha,df))


Using TensorFlow backend.


# Model

In [2]:
MP=False
Scope = 'Test'
cwd = os.getcwd()
# for Site in ['Illisarvik','FishIsland']:
Site='Illisarvik'
target='fch4'

# params['Loss']='mean_absolute_error'

if __name__ == '__main__':
    
    XVarriables=ModSelect(Scope,Site)
    prog = FloatProgress(min=0, max=len(XVarriables),description='Running:') # instantiate the bar
    display(prog) # display the bar
    try:
        shutil.rmtree(cwd+'/'+Site+'/'+target+'/')
    except:
        pass
    os.mkdir(cwd+'/'+Site+'/'+target+'/')
    for j in range(1,len(XVarriables)+1):
        if j == 1:
            Inputs = (Combos(XVarriables,j))
        else:
            Inputs = (Combos(XVarriables,j,Factors))
        i = 0
        for Input in Inputs:
            params = Dense.Params(Scope,target,MP)
            params['Dpath'] = cwd+'/'+Site+'/'
            params['Spath'] = params['Dpath']+'/'+target+'/'+str(j)+'_'+str(i)+'/'
            try:
                os.mkdir(params['Spath'])
            except:
                pass
            params['Sname'] = 'Y_'
            params['Inputs'] = Input
            params['Model'] = '+'.join(params['Inputs'])

            RST = RSTF.ReadStandardTimeFill(params['Dpath']+'ECData.csv',resample='2H')
            RST.Scale(params['target'],params['Inputs'])
            scaler_filename = "Y_scaler.save"
            joblib.dump(RST.YScaled, scaler_filename) 
            scaler_filename = "X_scaler.save"
            joblib.dump(RST.XScaled, scaler_filename) 
            y = RST.y*1.0
            X = RST.X*1.0

            params['N']=int(y.shape[0]/30)
#             mse = Test(params,X,y,RST.YScaled,RST.XScaled,pool)
            params['Memory'] = (math.floor(100/params['proc'])- 5/params['proc']) * .01
            Y_hat=[]
            y_true=[]
            X_true=[]
            index=[]
            ones=[]
            if MP == False:
                for k in range(params['K']):
                    results = Dense.TTV_Split(k,params,X,y)
                    Y_hat.append(RST.YScaled.inverse_transform(results[0]))
                    y_true.append(RST.YScaled.inverse_transform(results[1]))
                    X_true.append(RST.XScaled.inverse_transform(results[2]))
                    index.append(results[3])
                    ones.append(results[4])
            else:
                pool = Pool(processes=3,maxtasksperchild=75)
                for k,results in enumerate(pool.imap(partial(Dense.TTV_Split,params=params,X=X,y=y),range(params['K']))):
                    Y_hat.append(RST.YScaled.inverse_transform(results[0]))
                    y_true.append(RST.YScaled.inverse_transform(results[1]))
                    X_true.append(RST.XScaled.inverse_transform(results[2]))
                    index.append(results[3])
                    ones.append(results[4])
                pool.close()
                
#                 tf.keras.backend.clear_session()
            Y_hat = np.squeeze(np.asanyarray(Y_hat))
            y_true = np.squeeze(np.asanyarray(y_true))
            X_true = np.asanyarray(X_true)
            index = np.asanyarray(index)
            ones = np.asanyarray(ones)
            
            params['Memory'] = .95
            if MP == False:
                for k in range(1):
                     mse = Dense.Sort_outputs(k,params,Y_hat,y_true,X_true,index,ones)
            else:
                pool = Pool(processes=1,maxtasksperchild=75)
                for k,results in enumerate(pool.imap(partial(Dense.Sort_outputs,params=params,
                                                             Y_hat=Y_hat,y_true=y_true,X_true=X_true,index=index,ones=ones),
                                                     range(1))):
                     mse = results
                pool.close()
            
            if i == 0:
                Level = Stats(mse,j,i,params)
            else:
                Level = Level.append(Stats(mse,j,i,params))
            i += 1
            print(j+i/len(Inputs),j,i,len(Inputs))
            prog.value=j+i/len(Inputs)
        Min = Level.loc[Level['MSE']==Level['MSE']].min()
        T= 1#t(0.05,params['K'])
        Factors = Level.loc[Level['MSE']<=Min['MSE']+Min['SE']*T,'Model'].values
        Level.loc[Level['MSE']<=Min['MSE']+Min['SE']*T,'Performance']=1
#         print(Level)
        
        if j == 1:
            Records = Level
        else:
            Records = Records.append(Level)


Records = Records.reset_index()
plt.figure()
Min = Records.loc[Records['MSE']==Records['MSE']].min()
Records.loc[Records['MSE']<=Min['MSE']+Min['SE']*T,'Performance']=2
Records.loc[Records['MSE']==Min['MSE'],'Performance']=3
T= 1#t(0.05,params['K'])
Honorable = Records.loc[Records['Performance']==1]
Top = Records.loc[Records['Performance']==2]
Best = Records.loc[Records['Performance']==3]
plt.bar(Honorable.index,Honorable['MSE'],yerr=Honorable['SE'],color='blue')
plt.bar(Top.index,Top['MSE'],yerr=Top['SE'],color='green')
plt.bar(Best.index,Best['MSE'],yerr=Best['SE'],color='red')

A Jupyter Widget

Models:  [['PPFD_Avg'], ['wind_speed']]


InternalError: Blas GEMM launch failed : a.shape=(50, 1), b.shape=(1, 8), m=50, n=8, k=1
	 [[Node: dense_1/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](_arg_dense_1_input_0_0/_19, dense_1/kernel/read)]]
	 [[Node: loss/mul/_51 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_447_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'dense_1/MatMul', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py"

# Map Results

In [ ]:
# print(params['Sname'])
params['Spath']=(params['Dpath']+'/'+target+'/'+Best['index'].values[0]+'/')
# print(Best['Model'].values[0].split('+'))

RST.Scale(params['target'],Best['Model'].values[0].split('+'))
# print(Best['index'])
# for rm in Worst:
#     print(rm)
#     shutil.rmtree(params['Dpath']+'/'+target+'/'+rm)

In [ ]:
Fill = False
if Fill == True:
    X = RST.X_fill
else:
    X = RST.X
print(X.shape)
params['Sname']='Y_'
# params['Loss']='mean_absolute_error'
Y_fill = []
# Y_bar = []
MSE = []
for i in range(params['K']):
    params['iteration']=i
    Empty_Mod = Dense.Load_Model(params)
    Model = Dense.Load_Weights(Empty_Mod,params) 
    Y = RST.YScaled.inverse_transform(Model.predict(X).reshape(-1,1))
#     Y_bar.append(RST.YScaled.inverse_transform(Model.predict(np.median(X,axis=0)).reshape(-1,1)))
    if Fill == False:
        mse = (metrics.mean_squared_error(RST.y,Y))
        MSE.append(mse)
    Y_fill.append(Y)
Y_fill = np.asanyarray(Y_fill).mean(axis=-1)
# Y_bar = np.asanyarray(Y_bar).mean(axis=-1)
Y_fill_bar = Y_fill.mean(axis=0)
# Y_bar = Y_bar.mean(axis=0)
if Fill == False:
    MSE = np.asanyarray(MSE)
    CI = stats.t.ppf(1-0.025,i)*MSE.std()/(i)**.5
    print(CI)
    
print(Y_bar,Y_fill_bar.mean())

params['Sname']='Var'
params['iteration']=1
params['Loss']='Boot_Loss'
Empty_Mod = Dense.Load_Model(params)
Model = Dense.Load_Weights(Empty_Mod,params) 
YVar=YScaled.inverse_transform(Model.predict(X).reshape(-1,1))
YVar_bar=YScaled.inverse_transform(Model.predict(X.mean(axis=0)).reshape(-1,1))
X_back = np.squeeze(RST.XScaled.inverse_transform(X))

print(RST.YScaled.inverse_transform(RST.y).shape,np.squeeze(Y_fill_bar).shape)

Data = pd.DataFrame(data=X_back,columns=params['Inputs'])
Data[target] = np.squeeze(Y_fill_bar)
Data['True'] = RST.Master[target]#.YScaled.inverse_transform(RST.y)
Data['SE'] = 1/(params['K']-1)*((Y_fill-Y_fill_bar)**2).sum(axis=0)
Data['Var'] = np.squeeze(YVar)
Data['CI']=stats.t.ppf(1-0.025,params['K'])*(Data['SE'])**.5
Data['PI']=stats.t.ppf(1-0.025,params['K'])*((Data['Var']+Data['SE'])**.5) #the accuracy of our estimate with respect to the observed output

print(Data['CI'].mean())
print(Data[target].mean())
print(Data['True'].mean())
Data['Fill'] = Data['True'].fillna(Data[target])
print(Data['Fill'].mean())

# plt.figure(figsize=(8,7))
# Data = Data.sort_values(by='PPFD_Avg')
# Data.index = Data.PPFD_Avg


# plt.scatter(Data.index,Data['True'],edgecolor='black',facecolor='white')
# plt.plot(Data.index,Data[target],label=
#         params['target']+' Model\nRMSE: '+str(np.round(metrics.r2_score(Data['True'],
#                                                                    Data[params['target']])**2,3)))
# # plt.plot(Data.index,Data['Var'],label= params['target']+
# # ' Model\nRMSE: '+str(np.round(metrics.mean_squared_error(Data['True'],
# #                                                                    Data[params['target']])**2,3)))


# plt.fill_between(Data.index, Data[target]-Data['PI'], 
#                  Data[target]+Data['PI'],  color = 'green', alpha = 0.4, 
#                  label = '95% PI')
# plt.fill_between(Data.index, Data[target]-Data['CI'], 
#                  Data[target]+Data['CI'],  color = 'red', alpha = 0.4, 
#                  label = '95% CI')
# plt.legend()

# Sorting

# CI and PI

# The "Optimum" Sized Model

In [ ]:
# pool.close()
# Site = 'Illisarvik'#'FishIsland'#
Scope = 'Test'
cwd = os.getcwd()

# def Params(Func,Y,MP = True):
#     params = {}
#     params['proc']=3
#     if MP == False:
#         params['proc']=1
#     if Func == 'Full':
#         epochs = 200
#         K = 30
#         splits_per_mod = 1
#         N = np.linspace(200,20,10,dtype='int32')
#     elif Func == 'Test':
#         epochs = 200
#         K = 30
#         splits_per_mod = 1
#         N = np.linspace(70,10,5,dtype='int32')
#     N = np.repeat(N,K)
#     d = {'N':N.astype(int)}
#     Runs = pd.DataFrame(data=d)
#     Runs['MAE'] = 0.0
#     Runs['R2'] = 0.0
#     Runs['Model']=0
#     params['K'] = K
#     params['epochs'] = epochs
#     params['Y'] = Y
#     params['splits_per_mod'] = splits_per_mod
#     params['Save'] = {}
#     params['Save']['Weights']=False
#     params['Save']['Model']=False
    
#     return(Runs,params)


# MP=False

# if Scope == 'Full':
#     MP = True
# if __name__=='__main__'and MP==True:
#     pool = Pool(processes=3,maxtasksperchild=75)
# else:pool=None
    
# # for Site in ['Illisarvik','FishIsland']:
# Site='Illisarvik'
# FillVar = 'fco2'
# #     for FillVar in ['fco2','fch4']:
# Runs,params = MF.Params(Scope,FillVar,MP)
# FullModel = ModSelect(Scope,Site)
# print(FullModel)
# params['Dpath'] = cwd+'/'+Site+'/'
# params['Prelim_N']=True
# Best,Scores,ModelRuns = MF.FactorTest(params,FullModel,Runs)
# print(Best,Scores)
# Scores,ModelRuns = Best_Fill(Best,Runs,Scores,params)
# Scores.to_csv(params['Dpath']+FillVar+'/GapFillingSummary.csv')
# ModelRuns.to_csv(params['Dpath']+FillVar+'/GapFilled.csv')

# if __name__=='__main__'and MP==True:
#     pool.close()

In [ ]:
# Grp = Scores.groupby('Model').mean()
# Grp['SE'] = Scores[['Model','MAE']].groupby('Model').sem()
# # Grp['SE'] = Scores[['Key','MAE']].groupby('Key').sem()
# print(Grp)
# # plt.bar(Grp.index,Grp['MAE'],yerr=Grp['SE'])

In [ ]:
print('kitty')